In [17]:
_ARREST_KEY = "ARREST_KEY"

_ARREST_BORO = "ARREST_BORO"
_JURISDICTION_CODE = "JURISDICTION_CODE"
_OFFENSE_LEVEL = "LAW_CAT_CD"

_ARREST_DATE = "ARREST_DATE"
_ARREST_MONTH = "ARREST_MONTH" 

_ARREST_PRECINCT = "ARREST_PRECINCT"

_PERPETRATOR_RACE = "PERP_RACE"
_PERPETRATOR_SEX = "PERP_SEX"
_PERPETRATOR_AGE_GROUP="AGE_GROUP"

_INTERNAL_CLASSIFICATION="PD_CD"
_INTERNAL_CLASSIFICATION_DESCRIPTION="PD_DESC"
_THREE_DIGIT_INTERNAL_CLASSIFICATION="KY_CD"

_LAW_CODE="LAW_CODE"

_GEOGRAPHICAL_POSITION = "New Georeferenced Column"

_NYS_X_COORD = "X_COORD_CD"
_NYS_Y_COORD = "Y_COORD_CD"

_LATITUDE='Latitude'
_LONGITUDE='Longitude'


In [18]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC


from sklearn.metrics import accuracy_score, classification_report,confusion_matrix

from sklearn.svm import SVC, LinearSVC



In [19]:
df = pd.read_csv('./cleaned_data.csv')
df.head()


,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,Latitude,Longitude,ARREST_MONTH
0,FELONY ASSAULT,PL-1211200,F,Q,105,0,40.737043,-73.735514,January
1,FELONY ASSAULT,PL-1200502,F,B,48,0,40.855109,-73.892818,March
2,FELONY ASSAULT,PL-1200512,F,S,121,0,40.628967,-74.163275,May
3,FELONY ASSAULT,PL-1211200,F,Q,100,0,40.591980,-73.800066,June
4,RAPE,PL-1302503,F,M,14,0,40.753533,-73.994537,January


In [20]:
df[_ARREST_PRECINCT] = df[_ARREST_PRECINCT].astype('object')


In [21]:
df_num= df.select_dtypes(exclude='object')
df_obj= df.select_dtypes(include='object')

df_num.info()
df_obj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188682 entries, 0 to 188681
Data columns (total 3 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   JURISDICTION_CODE  188682 non-null  int64  
 1   Latitude           188682 non-null  float64
 2   Longitude          188682 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 4.3 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188682 entries, 0 to 188681
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   OFNS_DESC        188682 non-null  object
 1   LAW_CODE         188682 non-null  object
 2   LAW_CAT_CD       188682 non-null  object
 3   ARREST_BORO      188682 non-null  object
 4   ARREST_PRECINCT  188682 non-null  object
 5   ARREST_MONTH     188682 non-null  object
dtypes: object(6)
memory usage: 8.6+ MB


In [22]:
df_obj[_OFFENSE_LEVEL].unique()
#df_obj=df_obj.drop(columns=["OFNS_DESC","LAW_CODE"])# drop some columns to deal with long fit time
df_obj

,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,ARREST_MONTH
0,FELONY ASSAULT,PL-1211200,F,Q,105,January
1,FELONY ASSAULT,PL-1200502,F,B,48,March
2,FELONY ASSAULT,PL-1200512,F,S,121,May
3,FELONY ASSAULT,PL-1211200,F,Q,100,June
4,RAPE,PL-1302503,F,M,14,January
...,...,...,...,...,...,...
188677,PETIT LARCENY,PL-1552500,M,Q,109,September
188678,ASSAULT 3 & RELATED OFFENSES,PL-1200001,M,M,34,September
188679,BURGLARY,PL-1402501,F,M,23,September
188680,OTHER OFFENSES RELATED TO THEFT,PL-1651503,M,B,44,September


In [ ]:
df_obj= pd.get_dummies(df_obj, drop_first=True)

target = (df_obj['LAW_CAT_CD_M'])
df_obj = df_obj.drop(columns=['LAW_CAT_CD_M'],axis=1)
df_obj

In [ ]:
attr = pd.concat([df_obj,df_num],axis=1)


print(attr.shape[1])
attr.head()

1103


,OFNS_DESC_ADMINISTRATIVE CODES,OFNS_DESC_ALCOHOLIC BEVERAGE CONTROL LAW,OFNS_DESC_ANTICIPATORY OFFENSES,OFNS_DESC_ARSON,OFNS_DESC_ASSAULT 3 & RELATED OFFENSES,OFNS_DESC_BURGLAR'S TOOLS,OFNS_DESC_BURGLARY,OFNS_DESC_CANNABIS RELATED OFFENSES,OFNS_DESC_CHILD ABANDONMENT/NON SUPPORT 1,OFNS_DESC_CRIMINAL MISCHIEF & RELATED OF,...,ARREST_MONTH_February,ARREST_MONTH_January,ARREST_MONTH_July,ARREST_MONTH_June,ARREST_MONTH_March,ARREST_MONTH_May,ARREST_MONTH_September,JURISDICTION_CODE,Latitude,Longitude
0,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,0,40.737043,-73.735514
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,0,40.855109,-73.892818
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,0,40.628967,-74.163275
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,0,40.591980,-73.800066
4,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,0,40.753533,-73.994537


In [25]:
from sklearn.model_selection import train_test_split

# Split the data into training and test
attr_train, attr_test, target_train, target_test = train_test_split(attr, target, test_size=0.40, random_state=5)

In [ ]:
#svc=SVC(probability=False)

In [ ]:
linearSVC=LinearSVC()
#Using linear SVC because it takes less time to run

In [28]:
linearSVC.fit(attr_train,target_train)

LinearSVC()

In [29]:
#svc.fit(attr_train,target_train)

In [30]:
#target_pred=svc.predict(attr_test)


In [31]:
target_pred=linearSVC.predict(attr_test)


In [32]:
print('Model accuracy score with default hyperparameters: {0:0.4f}'. format(accuracy_score(target_test, target_pred)))
print()
print('Classification Report')
print(classification_report(target_test, target_pred))
print()
print('Confusion Matrix')
print(confusion_matrix(target_test, target_pred))

Model accuracy score with default hyperparameters: 1.0000

Classification Report
              precision    recall  f1-score   support

       False       1.00      1.00      1.00     32917
        True       1.00      1.00      1.00     42556

    accuracy                           1.00     75473
   macro avg       1.00      1.00      1.00     75473
weighted avg       1.00      1.00      1.00     75473


Confusion Matrix
[[32917     0]
 [    0 42556]]
